Hello and welcome to the WorldCover Preprocessing Notebook!

Create a python envionment with the Version 3.10:
i.E. :
conda create -n WC python=3.10
activate with:
conda activate WorldCover
conda install -c conda-forge rasterio geopandas shapely fiona pyproj gdal


load the border file using geopandas and check its CRS

In [1]:
# define vector data paths for swiss shape
DATA_PATH = "../../../data"
WORLDCOVER_PATH = DATA_PATH + "/preprocessing/worldcover/2020"

In [2]:
# Change to your environment variables for PROJ and GDAL to ensure proper functioning of geospatial libraries.
import os
os.environ["PROJ_LIB"] = r"C:\Users\alexa\anaconda3\envs\WC\Library\share\proj"
os.environ["GDAL_DATA"] = r"C:\Users\alexa\anaconda3\envs\WC\Library\share\gdal"

import geopandas as gpd
import rasterio

In [3]:
import geopandas as gpd
# Replace with your actual file path and layer name if necessary
gpkg_path = DATA_PATH  + "/base/swissBOUNDARIES3D/swissBOUNDARIES3D_1_5_LV95_LN02.gpkg"
border_gdf = gpd.read_file(gpkg_path)
print(border_gdf.crs)  # Check the CRS


c:\Users\jonas\anaconda3\envs\worldcover\lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files. Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()
c:\Users\jonas\anaconda3\envs\worldcover\lib\site-packages\pyogrio\geopandas.py:275: UserWarning: More than one layer found in 'swissBOUNDARIES3D_1_5_LV95_LN02.gpkg': 'tlm_hoheitsgrenze' (default), 'tlm_hoheitsgebiet', 'tlm_bezirksgebiet', 'tlm_landesgebiet', 'tlm_kantonsgebiet'. Specify layer parameter to avoid this warning.
  result = read_func(


EPSG:2056


Crate a buffer of 1 kilometers around the border

In [4]:
buffered_border_gdf = border_gdf.copy()
buffered_border_gdf["geometry"] = buffered_border_gdf.buffer(1000)


fill the holes in the border geometry (optional)

In [18]:
from shapely.geometry import Polygon, MultiPolygon

def fill_all_holes(geom):
    return Polygon(geom.exterior) if geom.type == 'Polygon' else MultiPolygon([Polygon(g.exterior) for g in geom.geoms])

buffered_border_gdf["geometry"] = buffered_border_gdf.geometry.apply(fill_all_holes)


C:\Users\jonas\AppData\Local\Temp\ipykernel_17984\323703650.py:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  return Polygon(geom.exterior) if geom.type == 'Polygon' else MultiPolygon([Polygon(g.exterior) for g in geom.geoms])


Save the buffered geopackage

In [ ]:
buffered_border_gdf.to_file(WORLDCOVER_PATH  + "/swiss_border_buffered_1km.gpkg", driver="GPKG")



load the raster files

In [ ]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob

# Replace this with your actual folder containing the 3 WorldCover rasters
raster_folder = WORLDCOVER_PATH  + "/tiles/"

# List all .tif files in the folder
raster_files = glob.glob(raster_folder + "*.tif") #

# Verify that three tiles were found
print(raster_files)


[]


open and merge the raster files

In [7]:
src_files_to_mosaic = [rasterio.open(fp) for fp in raster_files]
mosaic, mosaic_transform = merge(src_files_to_mosaic)


Copy metadata from one source raster

In [8]:
out_meta = src_files_to_mosaic[0].meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": mosaic_transform,
    "crs": src_files_to_mosaic[0].crs
})


Save the merged raster

In [ ]:
# Save as GeoTIFF (still in EPSG:4326)
out_fp = WORLDCOVER_PATH+"/worldcover_2020_merged.tif"
with rasterio.open(out_fp, "w", **out_meta) as dest:
    dest.write(mosaic)


change the CRS of the merged raster to EPSG:2056

In [10]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling


define input and output files

In [ ]:
# Input: merged raster still in EPSG:4326
src_fp = WORLDCOVER_PATH+"/worldcover_2020_merged.tif"

# Output: reprojected raster
dst_fp = WORLDCOVER_PATH+"/worldcover_2020_merged_epsg2056.tif"


Open the source raster and perform the reprojection.
This might take a while depending on the size of the raster and your computer performance

In [12]:
with rasterio.open(src_fp) as src:
    transform, width, height = calculate_default_transform(
        src.crs, "EPSG:2056", src.width, src.height, *src.bounds
    )

    kwargs = src.meta.copy()
    kwargs.update({
        "crs": "EPSG:2056",
        "transform": transform,
        "width": width,
        "height": height
    })

    with rasterio.open(dst_fp, "w", **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs="EPSG:2056",
                resampling=Resampling.nearest  # keep categorical values intact
            )


Verify if the transformation was sucessful by checking the CRS

In [13]:
with rasterio.open(dst_fp) as r:
    print(r.crs)
    print(r.count, r.width, r.height)


EPSG:2056
1 105382 51000


load merged and transformed raster and geopakage with the buffered border

In [ ]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# Paths
raster_path = WORLDCOVER_PATH + "/worldcover_2020_merged_epsg2056.tif"
mask_path = WORLDCOVER_PATH  + "/swiss_border_buffered_1km.gpkg"

# Read buffered border
mask_gdf = gpd.read_file(mask_path)
print(mask_gdf.crs)


EPSG:2056


 Convert mask geometries

In [15]:
mask_geom = [mask_gdf.union_all()]  # Combine if multiple polygons


Clip (mask) the raster

In [16]:
with rasterio.open(raster_path) as src:
    clipped_array, clipped_transform = mask(
        dataset=src,
        shapes=mask_geom,
        crop=True,
        nodata=0
    )
    clipped_meta = src.meta.copy()

clipped_meta.update({
    "driver": "GTiff",
    "height": clipped_array.shape[1],
    "width": clipped_array.shape[2],
    "transform": clipped_transform,
})

Save the clipped raster

In [ ]:
out_path = WORLDCOVER_PATH + "worldcover_2020_clipped.tif"
with rasterio.open(out_path, "w", **clipped_meta) as dest:
    dest.write(clipped_array)


Now you should have a clipped raster of WorldCover 2020 for Switzerland with a 1km buffer around the border! If that is not the case feel free to reach out to my email: alexander.ruehli@students.fhnw.ch so we can cry about it together :)